In [103]:
!pip install pytorch-lightning
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import tqdm
import pytorch_lightning as L
PATH = '/kaggle/input/yandex-cup-ml-23-nowcasting/ML Cup 2023 Weather/train/'
from pytorch_lightning import seed_everything
import random
seed=7
seed_everything(seed, workers=True)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [104]:
class SeasonDataset(data.Dataset):

    def __init__(self, list_of_files, season, in_seq_len=4, out_seq_len=12, mode='sequentially', rotate = 0, with_time=False):
        self.in_seq_len = in_seq_len
        self.out_seq_len = out_seq_len
        self.seq_len = in_seq_len + out_seq_len
        self.with_time = with_time
        self.__prepare_timestamps_mapping(list_of_files)
        self.__prepare_sequences(mode)
        self.rotate = rotate
        self.season = season

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        to_append = []
        data = []
        #targets = []
        for timestamp in self.sequences[index]:
            with h5py.File(self.timestamp_to_file[timestamp]) as d:
                #targets.append(np.array([1]))
                
                data.append(np.array([d[timestamp]['intensity']]))

                #data.append(np.array([d[timestamp]['intensity']]))

        data = np.array(data)
        #data = np.expand_dims(data, axis=1) ???
        targets = np.array([self.season])
        data[data == -1e6] = 0
        data[data == -2e6] = -1
        inputs = data[:self.in_seq_len]
        #targets = targets[self.in_seq_len:]
        if self.with_time:
            return (inputs, self.sequences[index][-1]), targets
        else:
            return inputs, targets

    def __prepare_timestamps_mapping(self, list_of_files):
        self.timestamp_to_file = {}
        for filename in list_of_files:
            with h5py.File(filename) as d:
                self.timestamp_to_file = {
                    **self.timestamp_to_file,
                    **dict(map(lambda x: (x, filename), d.keys()))
                }

    def __prepare_sequences(self, mode):
        timestamps = np.unique(sorted(self.timestamp_to_file.keys()))
        if mode == 'sequentially':
            self.sequences = [
                timestamps[index * self.seq_len: (index + 1) * self.seq_len]
                for index in range(len(timestamps) // self.seq_len)
            ]
        elif mode == 'overlap':
            self.sequences = [
                timestamps[index: index + self.seq_len]
                for index in range(len(timestamps) - self.seq_len + 1)
            ]
        else:
            raise Exception(f'Unknown mode {mode}')
        self.sequences = list(filter(
            lambda x: int(x[-1]) - int(x[0]) == (self.seq_len - 1) * 600,
            self.sequences
        ))

In [105]:
def prepare_month_loaders_class(train_batch_size=1):
    train_datasets = []
    val_datasets = []
    for i in range(1, 13):
        month = ''
        if i < 10:
            month = '0' + str(i)
        else:
            month = str(i)
        path = PATH + '2021-' + month + '-train.hdf5'
        month_dataset = SeasonDataset([path], season=0) if int(month) < 5 or int(month) > 8 else SeasonDataset([path], season=1)
        train_month, val_month = torch.utils.data.random_split(month_dataset, [0.8, 0.2])
        train_datasets.append(train_month)
        if int(month) >= 5 and int(month) <= 8:
            train_datasets.append(train_month)
        val_datasets.append(val_month)
    full_train_dataset = torch.utils.data.ConcatDataset([train_datasets[q] for q in range(len(train_datasets))])
    train_loader = data.DataLoader(full_train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4)
    val_loaders = [data.DataLoader(val_datasets[q], batch_size=1, shuffle=False, num_workers=4) for q in range(len(val_datasets))] 
    return train_loader, val_loaders

def evaluate_classification(model, valid_loader):
    correct_0 = 0
    correct_1 = 0
    incorrect_0 = 0
    incorrect_1 = 0
    default = 0
    for item in tqdm.tqdm(valid_loader):
        inputs, target = item
        inputs = inputs[0, :, 0, :, :]
        inputs = inputs.to(torch.device('cuda'))
        output = model(inputs)
        output = output.detach().cpu().numpy()
        if output > 0.6 and target == 1:
            correct_1 += 1
        elif output < 0.4 and target == 1:
            incorrect_1 += 1
        elif output < 0.4 and target == 0:
            correct_0 += 1
        elif output > 0.6 and target == 0:
            incorrect_0 += 1
        else:
            default += 1
            
    return correct_0, incorrect_0, correct_1, incorrect_1, default

In [106]:
class ClassModel(L.LightningModule):
    def __init__(self, num_kernels):
        super().__init__()
        
        self.activation = torch.relu
        self.conv1 = nn.Conv2d(4, num_kernels, kernel_size=3, padding=1) #32 252 252
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2) #96 126 126
        self.conv2 = nn.Conv2d(num_kernels, num_kernels, kernel_size=3, padding=1) #32 126 126
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2) #96 63 63
        self.conv3 = nn.Conv2d(num_kernels, num_kernels, kernel_size=3, padding=1) #32 63 63
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1) #32 32 32
        self.conv4 = nn.Conv2d(num_kernels, num_kernels, kernel_size=3, padding=1) #32 32 32
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2) #32 16 16
        self.conv5 = nn.Conv2d(num_kernels,num_kernels, kernel_size=3, padding=1) # 32 16 16
        self.maxpool5 = nn.MaxPool2d(kernel_size=2, stride=2) #32 8 8
        self.fc1 = nn.Linear(2048, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, X):
        output = self.sigmoid(self.fc2(self.activation(self.fc1(self.maxpool5(self.activation(self.conv5(self.maxpool4(self.activation(self.conv4(self.maxpool3(self.activation(self.conv3(
                self.maxpool2(self.activation(self.conv2(self.maxpool1(self.activation(self.conv1(X))))))))))))))).view(-1, 2048)))))
        return output
    
    def training_step(self, batch):
        x, y = batch
        x = x[0,:,0, :, :] #after this x.shape = [8, 252, 252]
        out = self.forward(x)
        loss = F.binary_cross_entropy(out.float(), y.float())
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-4)
        return optimizer
    

In [107]:
class_model = ClassModel(32)

In [108]:
class_train, class_val = prepare_month_loaders_class()

In [ ]:
class_model.current_lr = 3e-4
trainer = L.Trainer(
            max_epochs=1
        )
trainer.fit(class_model, class_train)

In [ ]:
class_model.current_lr = 5e-4
trainer = L.Trainer(
            max_epochs=1
        )
trainer.fit(class_model, class_train)

In [90]:
device = torch.device('cuda')
class_model.to(device)
for i in range(len(class_val)):
    print(evaluate_classification(class_model, class_val[i]))


100%|██████████| 55/55 [00:05<00:00, 10.77it/s]


(46, 1, 0, 0, 8)



100%|██████████| 49/49 [00:04<00:00, 12.20it/s]


(33, 1, 0, 0, 15)



100%|██████████| 49/49 [00:03<00:00, 12.42it/s]


...

(21, 4, 0, 0, 27)



100%|██████████| 52/52 [00:04<00:00, 12.12it/s]


(40, 0, 0, 0, 12)



100%|██████████| 55/55 [00:04<00:00, 12.09it/s]

(44, 3, 0, 0, 8)


In [91]:
torch.save(class_model, 'class_model_7e-4.pt')